In [ ]:
import os

import astropy.units as u
from astropy.table import Table
from astropy.time import Time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import schwimmbad

from thejoker.plot import plot_rv_curves
from thejoker.data import RVData
from thejoker.sampler import JokerParams, TheJoker, save_prior_samples

In [ ]:
allstar = Table.read("/Users/adrian/projects/twoface/twoface/db/tests/test-allStar.fits")
allvisit = Table.read("/Users/adrian/projects/twoface/twoface/db/tests/test-allVisit.fits")

In [ ]:
allstar['APOGEE_ID']

In [ ]:
APOGEE_ID = '2M00000222+5625359'
visits = allvisit[allvisit['APOGEE_ID'] == APOGEE_ID] # 4 observations, clear signal
star = allstar[allstar['APOGEE_ID'] == APOGEE_ID]

In [ ]:
t = Time(visits['JD'], format='jd', scale='utc')
rv = visits['VHELIO'] * u.km/u.s
err = np.sqrt(visits['VRELERR']**2 + 0.1**2) * u.km/u.s
data = RVData(t=t, rv=rv, stddev=err)

data.plot()

In [ ]:
params = JokerParams(P_min=8*u.day, P_max=8192*u.day)

In [ ]:
%%time 
with schwimmbad.MultiPool() as pool:
    joker = TheJoker(params, pool=pool)
    samples = joker.rejection_sample(data, n_prior_samples=16384*4) 

In [ ]:
lnP = np.log(samples['P'].value)
plt.plot(lnP, samples['K'], marker='.', ls='none')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,5))
t_grid = np.linspace(data.t.mjd.min()-50, data.t.mjd.max()+50, 1024)
fig = plot_rv_curves(samples, t_grid, rv_unit=u.km/u.s, data=data, ax=ax, 
                     n_plot=128, plot_kwargs=dict(color='#74a9cf', zorder=-100))
ax.set_xlim(t_grid.min(), t_grid.max())
ax.set_ylim(-75, -25)

In [ ]:
plt.hist(samples['v0'], bins=np.linspace(-100, 0, 64));
err = np.sqrt(0.1**2 + star['VERR']**2)
plt.axvline(star['VHELIO_AVG'], color='r')
plt.axvline(star['VHELIO_AVG']+err, color='r', linestyle='dashed')
plt.axvline(star['VHELIO_AVG']-err, color='r', linestyle='dashed')
plt.xlabel('heliocentric systemic velocity [km/s]')

---

In [ ]:
joker = TheJoker(params)
prior_samples = joker.sample_prior(16384) 

In [ ]:
filename = 'test-samples.hdf5'
if os.path.exists(filename):
    os.unlink(filename)
    
save_prior_samples(filename, prior_samples, u.km/u.s)

In [ ]:
%%prun -s cumulative
samples = joker.rejection_sample(data, prior_cache_file=filename)